In [1]:
import pandas as pd
import glob
import os

In [2]:
current_path = os.getcwd()
parent_directory = os.path.dirname(current_path)
source_path = os.path.join(parent_directory, 'data', 'raw')

file = glob.glob(os.path.join(source_path, "*.xlsx"))

df = pd.read_excel(file[0], index_col=None)
raw_shape = df.shape

In [3]:
df = df.sort_values(['Numer osobowy','Rodz. nieob./obecn.', 'Data od'])

df['Nowa_data_zakonczenia'] = df['Data do']
df['Dokonano_złączenia'] = 0

df.drop('Zm. dnia', axis=1, inplace=True)

In [4]:
def combine_lines(df):
    changes = 0
    for i in range(0, len(df)-1):
        if (i in df.index and 
            (i+1) in df.index and
            df.loc[i, 'Numer osobowy'] == df.loc[i+1, 'Numer osobowy'] and
            df.loc[i, 'Rodz. nieob./obecn.'] == df.loc[i+1, 'Rodz. nieob./obecn.'] and
            df.loc[i, 'Data do'] == df.loc[i+1, 'Data od'] - pd.Timedelta(days=1)):

            df.loc[i, 'Nowa_data_zakonczenia'] = df.loc[i+1, 'Data do']

            df.loc[i, 'Data do'] = df.loc[i+1, 'Data do']
            df.loc[i, 'Godziny nieobecności'] += df.loc[i+1, 'Godziny nieobecności']
            df.loc[i, 'Dni obecn./nieobecn.'] += df.loc[i+1, 'Dni obecn./nieobecn.']
            df.loc[i, 'Dni kalendarzowe'] += df.loc[i+1, 'Dni kalendarzowe']
            df.loc[i, 'Dokonano_złączenia'] = 'Tak'
            df.drop(index=[i+1], inplace=True)
            changes += 1
    
    return df, changes

In [5]:
total_changes = 0
while True:
    new_df, changes = combine_lines(df)
    
    if changes == 0:
        break

    total_changes += changes
    df = new_df
    df.reset_index(drop=True, inplace=True)

print(f'Combine rows - from {raw_shape[0]} to {df.shape[0]} rows.')
display(df)

Combine rows - from 24 to 8 rows. 16 combine rows.


,Numer osobowy,Rodz. nieob./obecn.,Nieobecność,Data od,Data do,Godziny nieobecności,Dni obecn./nieobecn.,Dni kalendarzowe,Grupa pracowników,Data ur.,Nowa_data_zakonczenia,Dokonano_złączenia
0,Osoba 1,NPZO,Op. Nad chor.Dzieck.,2023-03-06,2023-03-11,48,6,6,NaN,1985-11-18,2023-03-11,Tak
1,Osoba 1,UPZG,Płatny–MNiSzW z 12.10.06,2023-06-27,2023-07-14,112,14,16,NaN,1985-11-18,2023-07-14,Tak
2,Osoba 2,CHZ,Choroba zwykła,2023-03-20,2023-03-24,40,5,5,NNA,1989-01-26,2023-03-24,0
3,Osoba 2,CHZ,Choroba zwykła,2023-05-22,2023-06-02,80,10,10,NNA,1989-01-26,2023-06-02,Tak
4,Osoba 3,NPZO,Op. Nad chor.Dzieck.,2023-01-09,2023-12-04,248,31,32,NNA,1989-01-26,2023-12-04,Tak
5,Osoba 4,CHZ,Choroba zwykła,2023-09-28,2023-12-10,204,51,74,NNA,1965-11-27,2023-12-10,Tak
6,Osoba 5,CHZ,Choroba zwykła,2023-03-20,2023-03-24,40,5,5,NNA,1979-09-03,2023-03-24,0
7,Osoba 6,UPZG,Płatny–MNiSzW z 12.10.06,2023-06-27,2023-07-14,112,14,16,NaN,1985-05-04,2023-07-14,Tak
